# Representación Gráfica de Datos Estadísticos

In [ ]:
import pandas as pd   # Bibliotecapara el análisis de datos
import numpy as np   # Biblioteca para las operaciones matemáticas
import matplotlib.pyplot as plt   # Biblioteca de graficación

# Diccionario de Datos

Tesis: [Propiedades físicas de los aerosoles atmosféricos en Juriquilla, Querétaro.](http://132.248.9.195/ptd2019/junio/0790097/Index.html)

+ hora: hora de la medición en hora:minuto:segundo
+ fecha día de la medición
+ coedisp: coeficiente de dispersión de la luz incidente
+ abso: coeficiente de absorción de la luz incidente
+ albedo: coeficiente de albedo (porcentaje de radiación que la superficie refleja cuando un rayo de radiación incide sobre ella)
+ extinc: coeficiente de extinción, suma de los coeficientes de dispersión y absorción
+ cn: cantidad de carbono negro estimada

In [ ]:
# Lectura de datos
datos = pd.read_excel('Datos/todoslosmeses.xlsx')

In [ ]:
# Primeros 5 registros para ver estructura de la tabla
datos.head()

In [ ]:
# Columnas de la tabla
datos.columns

In [ ]:
# Reenombrando columnas
datos.columns = ['hora', 'fecha', 'coef_dispersion', 'coef_absorcion',
                 'coef_albedo', 'coef_extincion', 'cn']

In [ ]:
datos.head()

In [ ]:
# Info de la tabla
datos.info()

In [ ]:
# Método que devuelve los números espaciados uniformemente
np.linspace(0,1,11)

In [ ]:
# Indicamos el tamaño del intervalo con un arrange
np.arange(0,1.1,0.1)

In [ ]:
# Descriptivos de la tabla
datos.describe(percentiles=np.linspace(0,1,11))

Localizando el valor mínimo del cn:

In [ ]:
datos[datos.cn == datos.cn.min()]

Localizando la mediana del albedo:

In [ ]:
datos[datos['coef_albedo'] == datos['coef_albedo'].median()]

Localizando el valor máximo del carbono negro.

In [ ]:
datos[datos.cn == datos.cn.max()]

# Datos Nulos

NaN (nulos/NULL/...) es un marcador especial usado en diverosos lenguajes de programación para indicar que no existe un valor dentro de una base de datos.

<img src="Imagenes/rgde1.jpg">

Simplemente significa que la base de datos **NO cuenta con ese dato**.

Para saber si tenemos datos nulos utilizamos el método *isnull()*:

In [ ]:
datos.isnull().sum()

In [ ]:
# nulos.nul_values/datos.shape[0] * 100

In [ ]:
# Vemos el número de registros que tenemos
datos.shape[0]

In [ ]:
len(datos)

Generemos una tabla para ver qué porcentaje de cada variable corresponde a valores nulos:

In [ ]:
nulos = pd.DataFrame(datos.isnull().sum(), columns = ['nul_values'])

#se crea nueva columna de porcentaje
nulos['porcentaje'] = nulos.nul_values/datos.shape[0] * 100
nulos

Si queremos el número total de valores nulos bastará con:

In [ ]:
print("El número total de nulos es:", nulos.nul_values.sum())

Y si no tienes nada que hacer puedes agregar un último renglón de número total de nulos así:

In [ ]:
# Nos regresa la suma de los valores nulos
pd.DataFrame(nulos.nul_values.sum(),
             index=['total'],
             columns=['nul_values'])

In [ ]:
# Creamos una fila nueva
nulos.append(
    pd.DataFrame(data=nulos.nul_values.sum(),
                 index=['total'],
                 columns=['nul_values']))

Tenemos dos opciones para lidiar con los nulos: _remplazarlos por algún otro valor_ o simplemente _eliminar los registros_ que tengan valores nulos.

Para quitar nulos: 

In [ ]:
# datos.dropna(inplace=True)

Para reemplazarlos (en este caso lo remplaza por el promedio):

In [ ]:
# datos.fillna(datos.mean(), inplace=True)

¿Qué nos convendría más?

Vamos a ver:

In [ ]:
datos.isnull().sum()

Tenemos hasta 14 mil valores nulos en nuestras variables, si sólo tiraramos los valores nulos:

In [ ]:
t = (datos.isnull().sum().max())/ (len(datos)) * 100
print("Porcentaje aproximado de datos que perderíamos:", str(round(t,2)) + '%' )

En esta ocasión optaremos por rellenarlo con la mediana porque la mediana es más robusta a outliers que el promedio:

In [ ]:
# Datos para rellenar valores nulos
datos.fillna(datos.median(), inplace=True)

In [ ]:
datos.isnull().sum()

## ADVERTENCIA: NUNCA SE DEBEN REMPLAZAR LOS VALORES NULOS ANTES DEL ANÁLISIS ESTADÍSTICO DE LOS DATOS, MÁS ADELANTE VEREMOS PORQUÉ. EN ESTA CLASE LO HEMOS HECHO ASÍ PARA ILUSTRAR EL PUNTO

In [ ]:
# Distribuciones

plt.figure(figsize = (10,5))
datos.cn.hist(bins=100)

plt.xlim(-0.2, datos.cn.quantile(0.99))
plt.xlabel('carbono negro')
plt.ylabel(u'número de datos')
plt.title('Distribución del carbono negro acotado', fontsize=20,)
plt.xticks(fontsize = 18)
plt.yticks(fontsize = 16);

In [ ]:
# Histogramas de datos
datos.hist(bins=100, figsize=(15,10)) 
plt.tight_layout()  #ajusta automaticamente el padding

¿Qué son esos picos horribles en los coeficientes de extinción y en el albedo?

In [ ]:
datos.columns

In [ ]:
for col in datos.select_dtypes(exclude=["object", 'datetime']):   # Para columna en columna desde la columna 2 hasta la final
    print(col)   # Imprime tu iterador 
    datos[col].hist(bins=100,)   # Histograma de la columna con 100 intervalos 
    plt.xlim(datos[col].quantile(0.01),datos[col].quantile(0.99))   # Limite del eje x con respcto del 1 % al 99%
    plt.title(col,fontsize=20)
    plt.yticks(fontsize = 14)
    plt.xticks(fontsize = 14)
    plt.ylabel('Frecuencia', fontsize = 16)
    plt.show()
    print("------------------------------------------------------------------------")

## Correlaciones

In [ ]:
# Revisión de correlaciones
datos.corr(method='pearson')

In [ ]:
# Mapa de calor de las correlaciones
plt.figure(figsize = (10,10))
plt.imshow(datos.corr(), cmap='RdYlBu_r')
plt.colorbar()
plt.xticks(np.arange(5), datos.describe().columns, fontsize=8)
plt.yticks(np.arange(5), datos.describe().columns, fontsize=12);

Ahora hagamos lo mismo, pero con una biblioteca de visualización de datos más bonita:

In [ ]:
import seaborn as sbn

In [ ]:
corr = datos.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
sbn.heatmap(corr, annot=True, cmap='RdBu_r', vmin=-1,vmax=1, mask=mask);

In [ ]:
cov = datos.cov()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
sbn.heatmap(cov, annot=True, cmap='RdBu_r', mask=mask);

# Series de tiempo

In [ ]:
datos.head()

In [ ]:
datos['fecha']

In [ ]:
datos.groupby('fecha').mean().plot(figsize = (15,7));

In [ ]:
datos.hora.astype(str).str.split(":").str[0]

In [ ]:
datos["hora_buena"] = datos.hora.astype(str).str.split(":").str[0]

In [ ]:
datos.groupby('hora_buena').mean().plot(figsize = (15,5));